<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex09/patrick_ferreira/ex09_patrick_ferreira_175480.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Patrick de Carvalho Tavares Rezende Ferreira"
print(f'Meu nome é {nome}')

Meu nome é Patrick de Carvalho Tavares Rezende Ferreira


#  Exercício: Modelos de Linguagem com Bilhões de parâmetros e Implementação de Algoritmos de Decodificação

Neste exercicio iremos:
1. Avaliar a perplexidade do modelo GPT-2 (1.3B) ou GTP-J (6B) no dataset de testes do IMDB.
2. Ainda usando o GPT-2/J, iremos implementar algorimos de decodificação (sampling, top-k e top-k) e compara-los com os implementados pela função generate da HuggingFace. Esta análise será qualitativa.
3. Implementar uma função similar à torch.multinomial e checar se ela é igual ao do pytorch.

## Instalação/Carregamento dos pacotes

In [2]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes

     |████████████████████████████████| 148 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 62.5 MB 11.2 MB/s eta 0:00:01    |████████████▎                   | 24.0 MB 6.6 MB/s eta 0:00:06


In [3]:
import collections
import itertools
import functools
import math
import numpy as np
import os
import random
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from typing import List, Type

In [4]:
# Check which GPU we are using
!nvidia-smi

Wed Oct 26 17:49:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P0    N/A /  N/A |    245MiB /  2004MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


# Carregando o Modelo

In [7]:
# Para desenvolvimento, sugerimos usar o GPT menor, com 1.3B parametros.
model = transformers.GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B", device_map="auto", load_in_8bit=True)
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Se estiver usando o colab pro, conseguimos rodar um GPT-J de 6B de parametros.
# model = transformers.GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", torch_dtype=torch.float16, low_cpu_mem_usage=True)
# tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

# Testando geração com um prompt

In [8]:
prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)

verificacao = tokenizer(prompt, return_tensors="pt")

input_ids = tokenizer(prompt, return_tensors="pt").input_ids


In [ ]:
prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids.cuda(),
    do_sample=True,
    temperature=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

# Carregamento do dataset IMDB (conjunto de teste apenas)

Primeiro, fazemos download do dataset:

In [ ]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzf aclImdb.tgz

In [ ]:
def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_test = x_test_pos + x_test_neg

print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras validação:')
for x in x_test[:3]:
    print(x[:100])

print('3 últimas amostras validação:')
for x in x_test[-3:]:
    print(x[:100])




### Criando classe do dataset

class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int, vocab_size=1000):
        self.context_size = context_size
        try:
            self.x = np.load("x_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            self.y = np.load("y_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            print("Carregando dataset preprocessado")
        except Exception as e:
            # print("Excecao: ", e)
            print("Montando dataset")

            self.x = list()
            self.y = list()

            for text in tqdm(texts):
                tokens_key = tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids
                for i in range(0, len(tokens_key)-context_size-1, context_size):
                    self.x.append(tokens_key[i:i+context_size])
                    self.y.append(tokens_key[i+1:i+context_size+1])

            self.x = np.array(self.x)
            self.y = np.array(self.y)

            np.save("x_" + str(len(texts)) + ".npy", self.x)
            np.save("y_" + str(len(texts)) + ".npy", self.y)

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]).long(), torch.tensor(self.y[idx]).long(), torch.tensor([False] * self.context_size)

In [ ]:

def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 100

sample_train, target_token_ids, attention_mask = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
attention_mask = attention_mask.to(device)
logits = model(sample_train_gpu, attention_mask)

print("vocab_size: ", tokenizer.vocab_size)

print("logits shape: ", logits.shape)

my_perplexity = perplexity(logits=logits, target=target_token_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=6000)
print('Passou o no assert da perplexidade')

del sample_train_gpu
del target_token_ids
del logits


In [ ]:
context_size = 40

test_dataset = MyDataset(texts=x_test, tokenizer=tokenizer, context_size=context_size)
print(f'test examples: {len(test_dataset)}')

test_loader = DataLoader(test_dataset, batch_size=64, num_workers=1)